# CH05-BUILD THE NEURAL NETWORK

官网链接：https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

神经网络由layers/modules组成。`torch.nn`提供构成神经网络的层和模块。

每个modules都是`nn.Module`的子类，由layer构成。

本节将构建一个神经网络模型，对FashionMnist数据集做分类。

In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

## 1 确定设备

在GPU上训练模型，速度更快。

使用以下命令检查GPU是否可用：

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Using {} device'.format(device))

Using cuda device


## 2 定义模型

神经网络模型，继承自`nn.Module`类：

+ `__init__`初始化模型层

+ `forward`方法实现对输入数据的操作

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建 的实例NeuralNetwork，并将其移动到device，并打印其结构。

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


为了使用模型，我们将输入数据传递给它，这将执行模型的forward。

在输入上调用模型会返回一个 10 维张量，其中包含每个类的原始预测值。我们通过将其传递给nn.Softmax模块的实例来获得预测概率。

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


## 3 查看模型层变化

取minibatch=3, 大小为 28x28 的图像组成的小批量样本，看看通过网络传递该输入数据时，会发生什么。

In [7]:
input_image = torch.rand(3,28,28)

print(input_image.size())

torch.Size([3, 28, 28])


### 3.1 nn.Flatten()

我们初始化 nn.Flatten 层以将每个 2D 的 28x28 图像转换为 784 个像素值的连续数组（保持小批量维度（dim=0））

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)

print(flat_image.size())

torch.Size([3, 784])


### 3.2 nn.Lear()

线性层，它使用其存储的权重和偏差对输入做线性变换（保持小批量维度）

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)

hidden1 = layer1(flat_image)

print(hidden1.size())

torch.Size([3, 20])


### 3.3 nn.ReLU()

非线性激活是在模型的输入和输出之间创建复杂映射。

在线性变换之后应用非线性激活函数，帮助神经网络学习。

在这个模型中，我们在线性层之间使用了nn.ReLU，但是还有其他的非线性激活函数。

In [12]:
print(f"Before ReLU: {hidden1}\n\n")

hidden1 = nn.ReLU()(hidden1)

print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0585, -0.1585, -0.0222, -0.1794, -0.1152,  0.0181, -0.1770, -0.1645,
          0.2353,  0.5118,  0.3541,  0.0680,  0.2344,  0.1840, -0.2612,  0.2032,
          0.0201, -0.2444,  0.1717, -0.3305],
        [ 0.0336, -0.2431,  0.0490,  0.0894, -0.5817, -0.2275, -0.1509, -0.0694,
          0.0216,  0.2984,  0.1855, -0.3007,  0.1077,  0.0852,  0.1619,  0.1788,
         -0.4759, -0.1685, -0.0570, -0.4864],
        [ 0.0484, -0.1489,  0.1566,  0.3399, -0.2424, -0.1686,  0.1826, -0.6875,
         -0.0582,  0.4163,  0.2066, -0.2445,  0.3296,  0.1540,  0.0787,  0.2674,
         -0.3087, -0.3145, -0.1088, -0.5256]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0585, 0.0000, 0.0000, 0.0000, 0.0000, 0.0181, 0.0000, 0.0000, 0.2353,
         0.5118, 0.3541, 0.0680, 0.2344, 0.1840, 0.0000, 0.2032, 0.0201, 0.0000,
         0.1717, 0.0000],
        [0.0336, 0.0000, 0.0490, 0.0894, 0.0000, 0.0000, 0.0000, 0.0000, 0.0216,
         0.2984, 0.1855, 0.0000, 0.1077, 0.0852, 0.161

### 3.4 nn.Sequential

nn.Sequential 是一个有序的模块容器，输入数据按照定义时的模块顺序，依次通过所有模块。

可以使用顺序容器来组合一个像 seq_modules 这样的网络。

In [14]:
seq_modules = nn.Sequential(
    nn.Flatten(),
    # layer1,
    nn.Linear(in_features=28*28, out_features=20),
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

print(logits.shape)

torch.Size([3, 10])

### 3.5 nn.Softmax

`nn.Softmax`模块将 logits 值缩放 `[0, 1]`，该输出代表模型对每个类的预测概率。

dim参数指示输出值必须总和为 1 的 维度。

In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(pred_probab.max().item())

0.12905359268188477


## 4 模型参数

神经网络内的许多层都是参数化的，即在训练期间，这些参数（权重和偏差）会被优化。

`nn.Module`子类会自动跟踪模型对象中定义的所有参数，

模型中所有参数都可以通过模型的`parameters()`或`named_parameters()`方法访问。

In [19]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0189, -0.0119,  0.0202,  ..., -0.0101,  0.0341, -0.0219],
        [-0.0107, -0.0067,  0.0089,  ...,  0.0302, -0.0086,  0.0227]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0353,  0.0254], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0427, -0.0091,  0.0441,  ..., -0.0197,  0.0181,  0.0192],
        [ 0.0079,  0.0190,  0.0040,  ...,  0.0157, -0.0119, -0.0108]],
       device='cuda

更多关于`torch.nn`的API：https://pytorch.org/docs/stable/nn.html